In [5]:
import os
import pandas as pd
import numpy as np
import mne
from pathlib import Path
from scipy.signal import filtfilt, butter
import pywt

def preprocess_eeg(trial, user_id='s01'):
    def detrend_signal(raw):
        raw._data = mne.filter.detrend(raw.get_data(), order=1)
        return raw
    
    def apply_notch_filter(raw):
        raw.notch_filter(freqs=50)
        return raw
    
    def apply_ica(raw):
        ica = mne.preprocessing.ICA(n_components=15, random_state=97, max_iter='auto')
        ica.fit(raw)
        raw = ica.apply(raw)
        return raw
    
    def adaptive_filter(raw, reference_channel='EXG Channel 0'):
        if reference_channel not in raw.ch_names:
            raise ValueError(f"El canal de referencia {reference_channel} no está en los datos.")
        
        ecg_channel = raw.get_data(picks=reference_channel)
        eeg_channels = raw.get_data(picks='eeg')
        
        for i in range(eeg_channels.shape[0]):
            eeg_channels[i, :] -= np.mean(ecg_channel)
        
        raw._data[:eeg_channels.shape[0], :] = eeg_channels
        return raw

    
    def wavelet_thresholding(raw, wavelet='db4', level=4, threshold=0.2):
        eeg_data = raw.get_data(picks='eeg')
        for i in range(eeg_data.shape[0]):
            coeffs = pywt.wavedec(eeg_data[i, :], wavelet, level=level)
            coeffs_thresholded = [pywt.threshold(c, threshold * np.std(c), mode='soft') for c in coeffs]
            reconstructed = pywt.waverec(coeffs_thresholded, wavelet)

            # Ajustar la longitud si hay una diferencia de tamaño
            if reconstructed.shape[0] > eeg_data.shape[1]:
                reconstructed = reconstructed[:eeg_data.shape[1]]
            elif reconstructed.shape[0] < eeg_data.shape[1]:
                reconstructed = np.pad(reconstructed, (0, eeg_data.shape[1] - reconstructed.shape[0]), mode='constant')

            eeg_data[i, :] = reconstructed

        raw._data[:eeg_data.shape[0], :] = eeg_data
        return raw

    
    def bandpass_filter(data, lowcut, highcut, sfreq, order=4):
        nyquist = 0.5 * sfreq
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = butter(order, [low, high], btype='band')
        return filtfilt(b, a, data, axis=1)
    
    def segment_epochs(raw, epoch_length=4, overlap=0.5):
        sfreq = raw.info['sfreq']
        step = int(epoch_length * sfreq * (1 - overlap))
        epochs = []
        for start in range(0, raw.n_times - int(epoch_length * sfreq), step):
            end = start + int(epoch_length * sfreq)
            if end <= raw.n_times:
                epochs.append(raw.get_data()[:, start:end])
        return np.array(epochs) if epochs else None
    
    def extract_band_features(epoch, sfreq):
        bands = {
            "delta": (0.5, 4),
            "theta": (4, 8),
            "alpha": (8, 13),
            "beta": (13, 30),
            "gamma": (30, 45)
        }
        all_features = []
        for band_name, (low, high) in bands.items():
            filtered_epoch = bandpass_filter(epoch, low, high, sfreq)
            band_features = []
            for ch in filtered_epoch:
                power = np.sum(ch ** 2) / len(ch)
                rel_power = power / (np.sum(epoch ** 2) + 1e-12)
                std = np.std(ch)
                entropy = -np.sum((ch ** 2) * np.log2(ch ** 2 + 1e-12))
                min_val = np.min(ch)
                max_val = np.max(ch)
                diff1 = np.diff(ch)
                diff2 = np.diff(diff1)
                activity = np.var(ch)
                mobility = np.sqrt(np.var(diff1) / (activity + 1e-12))
                complexity = np.sqrt(np.var(diff2) / (np.var(diff1) + 1e-12))
                fft_values = np.abs(np.fft.rfft(ch))
                fft_mean = np.mean(fft_values)
                fft_top5 = np.sort(fft_values)[-5:]
                band_features.extend([power, rel_power, std, entropy, min_val, max_val, activity, mobility, complexity, fft_mean])
                band_features.extend(fft_top5)
            all_features.append(band_features)
        return np.concatenate(all_features)
    
    raw = detrend_signal(trial)
    raw = apply_notch_filter(raw)
    raw = apply_ica(raw)
    raw = adaptive_filter(raw)
    raw = wavelet_thresholding(raw)
    sfreq = raw.info['sfreq']
    epochs = segment_epochs(raw)
    if epochs is None:
        print(f"Advertencia: No se generaron épocas para {user_id}")
        return None
    all_epoch_features = []
    for epoch_idx, epoch in enumerate(epochs):
        epoch_features = extract_band_features(epoch, sfreq)
        epoch_id = f"{user_id}-epoch{epoch_idx + 1}"
        epoch_features_with_id = np.concatenate(([epoch_id], epoch_features))
        all_epoch_features.append(epoch_features_with_id)
    return np.array(all_epoch_features)

def process_openbci_data(input_folder, output_folder, sfreq=125):
    os.makedirs(output_folder, exist_ok=True)
    all_data = []  # Lista para almacenar todos los datos
    users = sorted(os.listdir(input_folder))
    for user in users:
        user_path = os.path.join(input_folder, user)
        if not os.path.isdir(user_path):
            continue
        print(f"📂 Procesando usuario: {user}")
        sessions = sorted(os.listdir(user_path))
        for session in sessions:
            if not session.endswith(".txt"):
                continue
            session_path = os.path.join(user_path, session)
            print(f"  📄 Sesión: {session}")
            df = pd.read_csv(session_path, delimiter='\t')
            eeg_channels = [f"EXG Channel {i}" for i in range(16)]
            eeg_data = df[eeg_channels].to_numpy().T
            info = mne.create_info(ch_names=eeg_channels, sfreq=sfreq, ch_types='eeg')
            raw = mne.io.RawArray(eeg_data, info)
            trial_features = preprocess_eeg(raw, user_id=f"{user}_{session}")
            if trial_features is not None:
                all_data.append(trial_features)  # Acumular los datos de todas las sesiones
    # Combina todos los datos en un solo DataFrame
    all_data_combined = np.vstack(all_data)
    # Definir las columnas
    columns = ['epoch_id'] + [f'feature_{i+1}' for i in range(all_data_combined.shape[1] - 1)]
    # Crear el DataFrame y guardarlo como CSV
    df_combined = pd.DataFrame(all_data_combined, columns=columns)
    output_file = os.path.join(output_folder, 'eeg_features_openbci_16.csv')
    df_combined.to_csv(output_file, index=False)
    print(f"    ✅ Guardado: {output_file}")
    
input_folder = "Señales_OpenBCI"
output_folder = "Señales_Procesadas"
process_openbci_data(input_folder, output_folder)


📂 Procesando usuario: TBCI-001
  📄 Sesión: Session_10_Video_10.txt
Creating RawArray with float64 data, n_channels=16, n_times=9379
    Range : 0 ... 9378 =      0.000 ...    75.024 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 s)

Fitting ICA to data using 16 channels (please be patient, this may take a while)
Selecting by number: 15 components


C:\Users\nicol\AppData\Local\Temp\ipykernel_31656\2750599098.py:20: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Fitting ICA took 0.3s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 16 PCA components
  📄 Sesión: Session_11_Video_11.txt
Creating RawArray with float64 data, n_channels=16, n_times=9379
    Range : 0 ... 9378 =      0.000 ...    75.024 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 s)

Fitting ICA to data using 16 channels (please be patient, this may take a while)
Selecting by numbe

C:\Users\nicol\AppData\Local\Temp\ipykernel_31656\2750599098.py:20: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


  📄 Sesión: Session_12_Video_12.txt
Creating RawArray with float64 data, n_channels=16, n_times=9378
    Range : 0 ... 9377 =      0.000 ...    75.016 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 s)

Fitting ICA to data using 16 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 0.1s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 1

...

  📄 Sesión: Session_9_Video_9.txt
Creating RawArray with float64 data, n_channels=16, n_times=9387
    Range : 0 ... 9386 =      0.000 ...    75.088 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 s)

Fitting ICA to data using 16 channels (please be patient, this may take a while)
Selecting by number: 15 components


C:\Users\nicol\AppData\Local\Temp\ipykernel_31656\2750599098.py:20: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Fitting ICA took 2.4s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 16 PCA components


c:\Python312\Lib\site-packages\sklearn\decomposition\_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


    ✅ Guardado: Señales_Procesadas\eeg_features_openbci_16.csv
